# Open Source GIS Pre-Processing Tutorial

## Table of Contents
1. [Create domain boundary shapefile](#Create-domain-boundary-shapefile)<br>
2. [Build geotiff from geogrid file](#Build-geotiff-from-geogrid-file)<br>
3. [Build routing stack](#Build-routing-stack)<br>
4. [Examine outputs of GIS pre-processor](#Examine-outputs-of-GIS-pre-processor)


## Create domain boundary shapefile

This tool takes an WRF Geogrid file and creates a single polygon shapefile that makes up the boundary of the domain of the M-grid (HGT_M, for example).

#### Get file paths
In this cell, variables are created that point to the file paths of the test data and a folder to store the output data created by these tools

In [1]:
import os
import Create_Domain_Boundary_Shapefile

cwd = os.getcwd()
data_folder = os.path.join(cwd, 'Croton_Lambert')
in_geogrid = os.path.join(data_folder, 'geo_em.d01.nc')
output_folder = os.path.join(cwd, 'Outputs')

Script initiated at Wed Sep 30 17:06:48 2020


#### Call help for the script on the command line
This is an example of the syntax for calling the Create Domain Boundary Shapefile tool on the command line. By following the tool with '-h', we are able to call the help arguement which explains the purpose of the tool, shows the different arguements we can use for this tool, as well as the descriptions for each arguement. 

It is not necessary to use the exclamation point on the command line, but it must be used to run this example in Jupyter. 

In [2]:
! python Create_Domain_Boundary_Shapefile.py -h

Script initiated at Wed Sep 30 17:06:49 2020
Script initiated at Wed Sep 30 17:06:49 2020
usage: Create_Domain_Boundary_Shapefile.py [-h] -i IN_NC -o OUT_DIR

This tool takes an WRF Geogrid file and creates a single polygon shapefile
that makes up the boundary of the domain of the M-grid (HGT_M, for example).

optional arguments:
  -h, --help  show this help message and exit
  -i IN_NC    Path to WPS geogrid (geo_em.d0*.nc) file or WRF-Hydro
              Fulldom_hires.nc file.
  -o OUT_DIR  Output directory.


#### Run the script on the command line
Now that we know what arguements are needed for this tool, we can call those arguements and run the tool. For this tool, we only need to specify the file path to the geogrid file and an output folder to save the result. 

The result of this tool is a shapefile that shows the geographic boundary of the domain of the geogrid file. 

When running this tool in Jupyter, we can use brackets around our variables to use with our parameters, however on the command line it is necessary to use a file path.

In [3]:
! python Create_Domain_Boundary_Shapefile.py -i {in_geogrid} -o {output_folder}

Script initiated at Wed Sep 30 17:06:51 2020
Script initiated at Wed Sep 30 17:06:51 2020
WPS netCDF projection identification initiated...
    Map Projection: Lambert Conformal Conic
    Using MOAD_CEN_LAT for latitude of origin.
    Using Standard Parallel 2 in Lambert Conformal Conic map projection.
    Geo-referencing step completed without error in  0.24 seconds.
    Created projection definition from input NetCDF GEOGRID file.
      ESRI Shapefile driver is available.
  Done producing output vector polygon shapefile in  0.02 seconds
  Output shapefile: C:\Users\casali\Documents\Projects\ForKevin\OpenSource_GIS_ProProcessing\Tutorials\Jupyter\Outputs\geo_em.d01_boundary.shp
Process completed in 0.27 seconds.


#### View the output boundary shapefile

Now that the domain boundary shapefile has been created, we want to see where it is on a map. This cell creates a map and adds the domain boundary as a layer. Use the map to explore the domain. A swipe feature allows the basemap to be changed to and from OpenStreetMap and satellite imagery. 

In [4]:
import json
import geopandas
from ipyleaflet import Map, GeoJSON, ScaleControl, FullScreenControl, basemaps, SplitMapControl, basemap_to_tiles, LayersControl

import warnings
warnings.filterwarnings("ignore")

# Setup display items
boundary_shp = os.path.join(output_folder,'geo_em.d01_boundary.shp')
b_shp = geopandas.read_file(boundary_shp)
b_shp = b_shp.to_crs(epsg=4326)

# Export vector to GeoJSON
b_json = os.path.join(output_folder, 'boundary.json')
b_shp.to_file(b_json, driver='GeoJSON')

# Read GeoJSON
with open(b_json, 'r') as f:
    data = json.load(f)
    
# Obtain vector center point
x = b_shp.geometry.centroid.x
y = b_shp.geometry.centroid.y
map_center = y[0], x[0]

# Instantiate map object
m = Map(center=(41.50, -73.73), zoom=10, scroll_wheel_zoom=True)

# Read GeoJSON
geo_json = GeoJSON(data=data, name='Domain boundary')
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(FullScreenControl())

# Define basemaps to swipe between
right_layer = basemap_to_tiles(basemap=basemaps.OpenStreetMap.Mapnik)
left_layer = basemap_to_tiles(basemap=basemaps.Esri.WorldImagery)

# Setup basemap swipe control
control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)
m.add_layer(geo_json)

control = LayersControl(position='topright')
m.add_control(control)

# Resize map
m.layout.width = '950px'
m.layout.height = '750px'

# Draw map
m

Map(center=[41.5, -73.73], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

## Build geotiff from geogrid file

This is a program to export >=2D variables from a WRF-Hydro input file (geogrid or Fulldom_hires) file to an output raster format, with all spatial and coordinate system metadata. If a 3-dimensional variable is selected, individual raster bands will be created in the output raster for each index in the 3rd dimension. If a 4-dimensional variable is selected, the first index in the 4th dimension will be selected and the variable will be treated as a 3-dimensional variable described above.

#### Run the script using Python
We will run this tool using Python rather than the command line method. The tool takes three input parameters: the geogrid file, a variable from that geogrid file, and a path for the output file. For this example, we will use the height in meters, "HGT_M", variable.

In [5]:
import rasterio
from matplotlib import pyplot
from Build_GeoTiff_From_Geogrid_File import build_geogrid_raster

in_geogrid = os.path.join(data_folder, 'geo_em.d01.nc')
output_folder = os.path.join(cwd, 'Outputs')
in_var = "HGT_M"
out_file = os.path.join(output_folder, f'{in_var}.tif')

build_geogrid_raster(in_geogrid, in_var, out_file)

WPS netCDF projection identification initiated...
    Map Projection: Lambert Conformal Conic
    Using MOAD_CEN_LAT for latitude of origin.
    Using Standard Parallel 2 in Lambert Conformal Conic map projection.
    Geo-referencing step completed without error in  0.13 seconds.
    X-dimension: 'west_east'.
    Y-dimension: 'south_north'.
    Reversing order of dimension 'south_north'
    Time dimension found: 'Time'.
      Time dimension size = 1.
    Dimensions and indices or slices on those dimensions:
        Time: 0
        south_north: slice(None, None, -1)
        west_east: slice(None, None, None)
    Size of array being sent to raster: (16, 15)
    GDAL Data type derived from input array: 6 (float32)
    Bands in output raster: 1
    Created GTiff format raster from HGT_M variable: C:\Users\casali\Documents\Projects\ForKevin\OpenSource_GIS_ProProcessing\Tutorials\Jupyter\Outputs\HGT_M.tif


#### View outputs
Now that the tool has completed, we want to take a look at the output. We will create another map and load the data as a layer. 

In [6]:
import matplotlib
from osgeo import gdal
from ipyleaflet import ImageOverlay
from jupyter_functions import cmap_options, show_raster_map

m2 = Map(center=map_center, zoom=10, scroll_wheel_zoom=True)

m2.add_control(ScaleControl(position='bottomleft'))
m2.add_control(FullScreenControl())
control = LayersControl(position='topright')
m2.add_control(control)

m2.layout.width = '950px'
m2.layout.height = '750px'

m2

Map(center=[41.47100098314024, -73.74365045481873], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [7]:
show_raster_map(out_file, m2, b_shp, output_folder)

## Build routing stack

This is a program to perform the full routing-stack GIS pre-processing for WRF-Hydro. The inputs will be related to the domain, the desired routing nest factor, and other options and parameter values. The output will be a routing stack zip file with WRF-Hydro domain and parameter files.

• Required:<br>
&emsp;-WRF GEOGRID file (.nc)<br>
&emsp;-High-resolution Elevation<br>
&emsp;&emsp;-Elevation file (Esri GRID, GeoTIFF, etc.)<br>
&emsp;&emsp;-Mosaic Dataset<br>

• Parameters<br>
&emsp;-Regridding Factor – nesting relationship of routing:land grids<br>
&emsp;-Minimum basin size (in routing grid cells)<br>
&emsp;-OVROUGHRTFAC – constant<br>
&emsp;-RETDEPRTFAC – constant<br>
&emsp;-LKSATFAC – constant<br>

• Optional:<br>
&emsp;-Station Locations (.csv)<br>
&emsp;-Lake Polygons (polygon feature class or .shp)<br>

#### Use command line to view the help message
This tool has many different parameters and possible configurations. Using the command line, we can take a look at the different arguements that can be used, if they are required or optional, and what their default values are.

In [8]:
os.chdir(cwd)
! python Build_Routing_Stack.py -h

Script initiated at Wed Sep 30 17:06:57 2020
usage: Build_Routing_Stack.py [-h] -i INGEOGRID [--CSV INCSV] [-b BASIN_MASK]
                              [-r RB_ROUTING] [-l IN_RESERVOIRS] -d INDEM
                              [-R CELLSIZE] [-t THRESHOLD] [-o OUT_ZIP_FILE]
                              [-O OVROUGHRTFAC_VAL] [-T RETDEPRTFAC_VAL]
                              [--starts CHANNEL_STARTS] [--gw GW_POLYS]

This is a program to perform the full routing-stack GIS pre-processingfor WRF-
Hydro. The inputs will be related to the domain, the desired routing nest
factor, and other options and parameter values. The output will be a routing
stack zip file with WRF-Hydro domain and parameter files.

optional arguments:
  -h, --help            show this help message and exit
  -i INGEOGRID          Path to WPS geogrid (geo_em.d0*.nc) file [REQUIRED]
  --CSV INCSV           Path to input forecast point CSV file [OPTIONAL]
  -b BASIN_MASK         Mask CHANNELGRID variable to forecast basi

#### Using Python to run the tool
We will begin by running a basic configuration of the tool using Python.

In [9]:
import shutil
from Build_Routing_Stack import GEOGRID_STANDALONE
from Build_Routing_Stack import varList2D
import wrfhydro_functions as wrfh

cwd = os.getcwd()
data_folder = os.path.join(cwd, 'Croton_Lambert')
output_folder = os.path.join(cwd, 'Outputs')

in_geogrid = os.path.join(data_folder, 'geo_em.d01.nc')
in_dem = os.path.join(data_folder, 'NED_30m_02b_Croton.tif')
out_zip = os.path.join(output_folder, 'croton_test.zip')
projdir = os.path.join(output_folder, 'scratch_dir')
regrid_factor = 10
routing_cells = 200

# Configure logging
logfile = out_zip.replace('.zip', '.log')
tee = wrfh.TeeNoFile(logfile, 'w')

# Create scratch directory for temporary outputs
projdir = os.path.join(output_folder, 'scratchdir')
projdir = os.path.abspath(projdir)
if os.path.exists(projdir):
    shutil.rmtree(projdir)
os.makedirs(projdir)

GEOGRID_STANDALONE(inGeogrid=in_geogrid, regridFactor=regrid_factor, inDEM=in_dem,
                   projdir=projdir, threshold=routing_cells, out_zip=out_zip, varList2D=varList2D )

tee.close()
del tee

  Reach-based routing files will not be created.
WPS netCDF projection identification initiated...
    Map Projection: Lambert Conformal Conic
    Using MOAD_CEN_LAT for latitude of origin.
    Using Standard Parallel 2 in Lambert Conformal Conic map projection.
    Geo-referencing step completed without error in  0.10 seconds.
  Building sub-grid of model grid.
    Original grid spacing dx=1000.0, dy=-1000.0
    Original grid size: rows=16, cols=15
    New grid spacing: dx=100.0, dy=-100.0
    New dimensions: rows=160, cols=150
    Created projection definition from input NetCDF GEOGRID file.
    Proj4: +proj=lcc +lat_0=41.4710083007813 +lon_0=-97 +lat_1=30 +lat_2=60 +x_0=0 +y_0=0 +R=6370000 +units=m +no_defs
    Coarse grid GeoTransform: 1841998.7848425123 1000.0 0 278496.6013001092 0 -1000.0
    Coarse grid extent [Xmin, Ymin, Xmax, Ymax]: [1841998.7848425123, 262496.6013001092, 1856998.7848425123, 278496.6013001092]
    Fine grid extent [Xmin, Ymin, Xmax, Ymax]:   [1841998.78484251

### Examine outputs of GIS pre-processor

This tool takes the output zip file from the Process Geogrid script and creates a raster from each output netCDF file. The Input should be a .zip file that was created using the WRF Hydro pre-processing tools. The tool will create the folder which will contain the results if that folder does not already exist.

In [10]:
from Examine_Outputs_of_GIS_Preprocessor import examine_outputs
from wrfhydro_functions import ZipCompat

raster_outputs = os.path.join(output_folder, "Raster_Outputs")
if not os.path.exists(raster_outputs):
    os.mkdir(raster_outputs)

ZipCompat(out_zip).extractall(raster_outputs)
examine_outputs(raster_outputs, skipfiles=[])

  GeoTransform: (1841998.7848425123, 100.0, 0.0, 278496.6013001092, 0.0, -100.0)
  DX: 100.0
  DY: 100.0
  PROJ.4 string: +proj=lcc +lat_0=41.4710083007813 +lon_0=-97 +lat_1=30 +lat_2=60 +x_0=0 +y_0=0 +R=6370000 +units=m +no_defs
    File Created: C:\Users\casali\Documents\Projects\ForKevin\OpenSource_GIS_ProProcessing\Tutorials\Jupyter\Outputs\Raster_Outputs\CHANNELGRID.tif
    File Created: C:\Users\casali\Documents\Projects\ForKevin\OpenSource_GIS_ProProcessing\Tutorials\Jupyter\Outputs\Raster_Outputs\FLOWDIRECTION.tif
    File Created: C:\Users\casali\Documents\Projects\ForKevin\OpenSource_GIS_ProProcessing\Tutorials\Jupyter\Outputs\Raster_Outputs\FLOWACC.tif
    File Created: C:\Users\casali\Documents\Projects\ForKevin\OpenSource_GIS_ProProcessing\Tutorials\Jupyter\Outputs\Raster_Outputs\TOPOGRAPHY.tif
    File Created: C:\Users\casali\Documents\Projects\ForKevin\OpenSource_GIS_ProProcessing\Tutorials\Jupyter\Outputs\Raster_Outputs\RETDEPRTFAC.tif
    File Created: C:\Users\casali

Now we want to take a look at the output rasters. Below, we utilize a Jupyter widget to choose each raster from a drop down menu. Take a look at some of the outputs.

In [11]:
import ipywidgets as widgets
from ipywidgets import interact

def see_raster(x):
    src = rasterio.open(os.path.join(raster_outputs, f"{x}.tif"))
    cmap, norm = cmap_options(x)
    if x in ['TOPOGRAPHY']:
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', norm=norm, interpolation='nearest', vmin=0)
    else:
        pyplot.imshow(src.read(1), cmap=cmap,  aspect='auto', norm=norm, interpolation='nearest')
    cbar = pyplot.colorbar()
    
    # Keep the automatic aspect while scaling the image up in size
    fig = pyplot.gcf()
    w, h = fig.get_size_inches()
    fig.set_size_inches(w * 1.75, h * 1.75)
    
    # Show image
    pyplot.show()

in_raster = widgets.Dropdown(
    options=[('Basin', 'BASIN'), ('Basin mask', 'basn_msk'), ('Channel grid', 'CHANNELGRID'), ('Flow accumulation', 'FLOWACC'),
            ('Flow direction', 'FLOWDIRECTION'), ('Forecast points', 'frxst_pts'), ('Lake grid', 'LAKEGRID'),
            ('Land use', 'landuse'), ('Latitude', 'LATITUDE'), ('LKSATFAC', 'LKSATFAC'), ('Longitude', 'LONGITUDE'),
            ('OVROUGHRTFAC', 'OVROUGHRTFAC'), ('RETDEPRTFAC', 'RETDEPRTFAC'), ('Stream order', 'STREAMORDER'),
            ('Topography', 'TOPOGRAPHY')],
    value='FLOWACC',
    description='Variable:')

interact(see_raster, x=in_raster)

interactive(children=(Dropdown(description='Variable:', index=3, options=(('Basin', 'BASIN'), ('Basin mask', '…

<function __main__.see_raster(x)>

We can also look at the raster data projected onto a map. Use the dropdown widget below the map to add rasters as layers to the map. Use the menu in the top right corner of the map to turn layers on and off. 

In [12]:
m3 = Map(center=map_center, zoom=10, scroll_wheel_zoom=True)

m3.add_control(ScaleControl(position='bottomleft'))
m3.add_control(FullScreenControl())
control = LayersControl(position='topright')
m3.add_control(control)

m3.layout.width = '950px'
m3.layout.height = '750px'

m3

Map(center=[41.47100098314024, -73.74365045481873], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [13]:
def f(x):
    r_path = os.path.join(raster_outputs, f"{x}.tif")
    show_raster_map(r_path, m3, b_shp, output_folder)

in_raster = widgets.Dropdown(
    options=[('Basin', 'BASIN'), ('Flow accumulation', 'FLOWACC'), ('Flow direction', 'FLOWDIRECTION'), 
             ('Land use', 'landuse'), ('Stream order', 'STREAMORDER'), ('Topography', 'TOPOGRAPHY')],
    value='FLOWDIRECTION',
    description='Variable:',
)
interact(f, x=in_raster)

interactive(children=(Dropdown(description='Variable:', index=2, options=(('Basin', 'BASIN'), ('Flow accumulat…

<function __main__.f(x)>